# 1. Combinar archivos 
#### Archivo de enrgia extraida con archivo de empresas

In [15]:
import pandas as pd
import glob
import re
import os
from pathlib import Path

# === CONFIGURACIÓN ===
CENTRALES_FILE = './data/1_empresas.xlsx'
INPUT_PATTERN = './downloads/extracted_ingresos_c_iny_*.xlsx'
OUTPUT_PREFIX = './downloads/ingresos_centrales_'

# === FUNCIONES AUXILIARES ===

def detectar_fila_encabezado(df):
    for i, row in df.iterrows():
        if any(str(cell).strip().upper() == "CENTRAL" for cell in row):
            return i
    return 0

def normalizar_nombre(nombre, nombres_centrales, alias):
    x = str(nombre).strip()
    if x in alias:
        return alias[x]
    
    # Limpieza más profunda para coincidencias flexibles
    x_clean = re.sub(r'\W+', '', x).upper()
    
    for k in nombres_centrales:
        k_clean = re.sub(r'\W+', '', k).upper()
        if k_clean == x_clean:
            return k
    
    # Casos especiales para Aguaí
    if "AGUAI" in x_clean or "AGUAÍ" in x_clean:
        if "AUTOPRODUCTOR" in x_clean:
            return "Aguai (Autoproductor)"
        return "Aguaí Energia"
    
    return x

# === MAPA DE ALIAS DE CENTRALES ===
alias = {
    "Kanata en Arocagua": "Kanata ARO",
    "Kanata en Valle Hermoso": "Kanata VHE",
    "Misicuni en Arocagua": "Misicuni ARO",
    "Misicuni en Valle Hermoso": "Misicuni VHE",
    "Yunchara": "Yunchara",
    "Aguaí Energía": "Aguaí Energia",
    "AGUAÍ ENERGÍA S.A.": "Aguaí Energia",
    "Santa Cruz (Aguaí)": "Santa Cruz (Aguaí)",
    "RÍO ELÉCTRICO S.A.": "RIO ELECTRICO S.A.",
    "CHACO ENERGÍAS S.A.": "CHACO ENERGIAS S.A.",
    "RIOELEC S.A.": "RIO ELECTRICO S.A.",
}

# === PROCESAMIENTO DE ARCHIVOS ===

def procesar_archivos():
    try:
        df_centrales = pd.read_excel(CENTRALES_FILE)
        df_centrales['CENTRAL'] = df_centrales['CENTRAL'].astype(str).str.strip()

        if not {'CENTRAL', 'GENERADOR', 'TECNOLOGIA'}.issubset(df_centrales.columns):
            print("Error: El archivo debe contener columnas 'CENTRAL', 'GENERADOR' y 'TECNOLOGIA'")
            return

        mapeo_generadores = dict(zip(df_centrales['CENTRAL'], df_centrales['GENERADOR']))
        mapeo_tecnologia = dict(zip(df_centrales['CENTRAL'], df_centrales['TECNOLOGIA']))
        nombres_centrales = set(df_centrales['CENTRAL'])

        # Asegurar centrales de Aguaí en el mapeo
        for central_aguai in ["Aguaí Energia", "Aguai (Autoproductor)"]:
            if central_aguai not in mapeo_generadores:
                mapeo_generadores[central_aguai] = "AGUAÍ ENERGÍA S.A."
                mapeo_tecnologia[central_aguai] = "Biomasa"

    except Exception as e:
        print(f"Error al cargar archivo de mapeo {CENTRALES_FILE}: {str(e)}")
        return

    for input_file in glob.glob(INPUT_PATTERN):
        file_number = re.search(r'extracted_ingresos_c_iny_(\d+)\.xlsx', input_file)
        if not file_number:
            continue

        file_number = file_number.group(1)
        output_file = f"{OUTPUT_PREFIX}{file_number}.xlsx"

        if os.path.exists(output_file):
            print(f"[Omitido] {output_file} ya existe.")
            continue

        try:
            df_raw = pd.read_excel(input_file, header=None)
            start_row = detectar_fila_encabezado(df_raw)
            df = pd.read_excel(input_file, skiprows=start_row)
            df.columns = [str(col).strip() for col in df.columns]

            central_col = next((c for c in df.columns if 'central' in c.lower() or 'agente' in c.lower()), None)
            if central_col and central_col != 'CENTRAL':
                df = df.rename(columns={central_col: 'CENTRAL'})
            if 'CENTRAL' not in df.columns:
                print(f"[Error] No se encontró columna 'CENTRAL' en {input_file}")
                continue

            df['CENTRAL'] = df['CENTRAL'].astype(str).str.strip()

            # Eliminación de filas basura (incluyendo totales de Aguaí)
            pattern_basura = r'TOTAL|TOTALES|Nota|Tipo de cambio|nan|CARGOS POR INYECCIONES|TOTAL\s*-\s*AGUAI'
            df = df[~df['CENTRAL'].str.contains(pattern_basura, case=False, na=True, regex=True)]
            df = df[~df['CENTRAL'].str.match(r'^\d{4}-\d{2}-\d{2}', na=False)]
            df = df[~df['CENTRAL'].str.upper().str.contains('CENTRAL\s*ENERGIA|POTENCIA', na=False)]
            df = df[df['CENTRAL'].notna() & (df['CENTRAL'] != '')]

            # Identificación de filas válidas
            df['CENTRAL_CLEAN'] = df['CENTRAL'].str.strip().str.upper()
            centrales_validas = set(x.upper() for x in nombres_centrales)
            first_valid_idx = df[df['CENTRAL_CLEAN'].isin(centrales_validas)].index.min()
            if pd.isna(first_valid_idx):
                print(f"[Error] No se encontraron centrales válidas en {input_file}")
                continue
            df = df.loc[first_valid_idx:].copy()

            # Normalización y mapeo
            df['CENTRAL_NORMALIZADA'] = df['CENTRAL'].apply(
                lambda x: normalizar_nombre(x, nombres_centrales, alias)
            )
            df['GENERADOR'] = df['CENTRAL_NORMALIZADA'].map(mapeo_generadores)
            df['TECNOLOGIA'] = df['CENTRAL_NORMALIZADA'].map(mapeo_tecnologia)

            # Forzar presencia de ambas centrales Aguaí
            aguai_centrales = ["Aguaí Energia", "Aguai (Autoproductor)"]
            for central in aguai_centrales:
                if central not in df['CENTRAL_NORMALIZADA'].values:
                    nueva_fila = {
                        'CENTRAL': central,
                        'CENTRAL_NORMALIZADA': central,
                        'GENERADOR': "AGUAÍ ENERGÍA S.A.",
                        'TECNOLOGIA': "Biomasa"
                    }
                    for col in df.columns:
                        if 'kW' in col or 'kWh' in col:
                            nueva_fila[col] = 0.0
                    df = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)

            # Procesamiento numérico
            for col in df.columns:
                if 'kW' in col or 'kWh' in col:
                    df[col] = (
                        df[col]
                        .astype(str)
                        .str.replace(',', '')
                        .str.replace(' ', '')
                        .replace('nan', None)
                        .astype(float)
                    )

            # ... código existente ...

            columnas_finales = ['CENTRAL_NORMALIZADA', 'GENERADOR', 'TECNOLOGIA'] + [
                c for c in df.columns if c not in ['CENTRAL', 'CENTRAL_NORMALIZADA', 'CENTRAL_CLEAN', 'GENERADOR', 'TECNOLOGIA']
            ]
            df_final = df[columnas_finales].rename(columns={'CENTRAL_NORMALIZADA': 'CENTRAL'})


# ... resto del código existente ...
            # Guardar el DataFrame final
            df_final.to_excel(output_file, index=False)

            print(f"[OK] {input_file} → {output_file}")

            # Detectar centrales sin mapeo
            faltantes = df_final[df_final['GENERADOR'].isna()]['CENTRAL'].unique()
            if len(faltantes) > 0:
                print(f"  ⚠️ {len(faltantes)} centrales sin GENERADOR: {faltantes[:3]}{'...' if len(faltantes) > 3 else ''}")

        except Exception as e:
            print(f"[Error] {input_file}: {str(e)}")
            try:
                Path('./errors').mkdir(exist_ok=True)
                df.to_excel(f"./errors/ERROR_{file_number}.xlsx", index=False)
            except Exception as inner_e:
                print(f"Error al guardar archivo de error: {inner_e}")

# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    print("🔄 Iniciando procesamiento de archivos...")
    procesar_archivos()
    print("✅ Proceso completado.")

<>:112: SyntaxWarning: invalid escape sequence '\s'
<>:112: SyntaxWarning: invalid escape sequence '\s'
C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_48132\4005102116.py:112: SyntaxWarning: invalid escape sequence '\s'
  df = df[~df['CENTRAL'].str.upper().str.contains('CENTRAL\s*ENERGIA|POTENCIA', na=False)]


🔄 Iniciando procesamiento de archivos...
[OK] ./downloads\extracted_ingresos_c_iny_0123.xlsx → ./downloads/ingresos_centrales_0123.xlsx
[OK] ./downloads\extracted_ingresos_c_iny_0124.xlsx → ./downloads/ingresos_centrales_0124.xlsx
  ⚠️ 1 centrales sin GENERADOR: ['Yunchará']
[OK] ./downloads\extracted_ingresos_c_iny_0125.xlsx → ./downloads/ingresos_centrales_0125.xlsx
[OK] ./downloads\extracted_ingresos_c_iny_0223.xlsx → ./downloads/ingresos_centrales_0223.xlsx
[OK] ./downloads\extracted_ingresos_c_iny_0224.xlsx → ./downloads/ingresos_centrales_0224.xlsx
  ⚠️ 1 centrales sin GENERADOR: ['Yunchará']
[OK] ./downloads\extracted_ingresos_c_iny_0225.xlsx → ./downloads/ingresos_centrales_0225.xlsx
[OK] ./downloads\extracted_ingresos_c_iny_0323.xlsx → ./downloads/ingresos_centrales_0323.xlsx
  ⚠️ 5 centrales sin GENERADOR: ['San Borja' 'Rurrenabaque' 'Yucumo']...
[OK] ./downloads\extracted_ingresos_c_iny_0324.xlsx → ./downloads/ingresos_centrales_0324.xlsx
[OK] ./downloads\extracted_ingresos_

# 2. Merge archivos energia

In [16]:
import pandas as pd
import glob
from datetime import datetime

def consolidar_en_formato_largo():
    archivos = sorted(glob.glob('./downloads/ingresos_centrales_*.xlsx'))
    if not archivos:
        print("No se encontraron archivos.")
        return

    registros = []

    for archivo in archivos:
        try:
            # Extraer mes y año del nombre del archivo
            periodo = archivo.split('_')[-1].split('.')[0]
            mes = int(periodo[:2])
            año = 2000 + int(periodo[2:])
            fecha = datetime(año, mes, 1)

            df = pd.read_excel(archivo)

            # Verificar columnas esenciales
            if 'CENTRAL' not in df.columns:
                print(f"Omitido: {archivo} no tiene columna CENTRAL.")
                continue

            columnas_necesarias = ['CENTRAL', 'GENERADOR']
            columnas_datos = [col for col in df.columns if col not in columnas_necesarias + ['TECNOLOGIA']]
            
            # Si no hay columnas de datos, omitir
            if not columnas_datos:
                print(f"Omitido: {archivo} no tiene columnas de datos.")
                continue

            # Si no existe columna TECNOLOGIA, crear una con NaN
            if 'TECNOLOGIA' not in df.columns:
                df['TECNOLOGIA'] = None

            # Derretir (melt) el DataFrame para convertir a formato largo
            temp = pd.melt(
                df,
                id_vars=['CENTRAL', 'GENERADOR', 'TECNOLOGIA'],
                value_vars=columnas_datos,
                var_name='VARIABLE',
                value_name='VALOR'
            )
            
            temp['FECHA'] = fecha
            registros.append(temp)

        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
            continue

    if not registros:
        print("No se pudo consolidar ningún archivo válido.")
        return

    df_largo = pd.concat(registros, ignore_index=True)
    df_largo = df_largo[['FECHA', 'CENTRAL', 'GENERADOR', 'TECNOLOGIA', 'VARIABLE', 'VALOR']]

    df_largo.to_excel("./preprocess/serie_temporal_ingresos.xlsx", index=False)
    print("Consolidación completada en formato largo.")
    print(f"Filas totales: {len(df_largo)}")
    print("Archivo guardado como 'serie_temporal_ingresos.xlsx'")

if __name__ == "__main__":
    consolidar_en_formato_largo()

Consolidación completada en formato largo.
Filas totales: 5068
Archivo guardado como 'serie_temporal_ingresos.xlsx'


# 3. Creacion de serie de tiempo de energia

In [23]:
import pandas as pd

# Leer archivo
df = pd.read_excel("./preprocess/serie_temporal_ingresos.xlsx")

# Convertir FECHA y eliminar filas sin fecha válida
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce', unit='d')
df = df.dropna(subset=['FECHA'])

# Limpiar espacios en columnas clave
df['CENTRAL'] = df['CENTRAL'].astype(str).str.strip()
df['GENERADOR'] = df['GENERADOR'].astype(str).str.strip()
df['TECNOLOGIA'] = df['TECNOLOGIA'].astype(str).str.strip()
df['VARIABLE'] = df['VARIABLE'].astype(str).str.strip()

# Filtrar sólo las variables de interés exactas (con mayúsculas y espacios)
df = df[df['VARIABLE'].isin([
    "Energía KWh",
    'Ingresos Energía USD',
    'Ingresos Renovables USD',
    'Ingresos Potencia USD'
])]

# Asegurar que VALOR sea numérico
df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce')
df = df.dropna(subset=['VALOR'])

# Crear MES_ANIO
df['MES_ANIO'] = df['FECHA'].dt.strftime('%m%Y')

# Reemplazar valores faltantes en AGENTE y EMPRESA con forward fill
df['CENTRAL'] = df['CENTRAL'].replace('nan', pd.NA).fillna(method='ffill')
df['GENERADOR'] = df['GENERADOR'].replace('nan', pd.NA).fillna(method='ffill')

# Crear columna pivote combinando variable y mes_año (con espacios y mayúsculas)
df['COLUMNA'] = df['VARIABLE'] + ' ' + df['MES_ANIO']

# Pivotear el DataFrame
df_pivot = df.pivot_table(index=['CENTRAL', 'GENERADOR', "TECNOLOGIA"], columns='COLUMNA', values='VALOR', aggfunc='first')

# Función para extraer año y mes para ordenar correctamente
def extraer_anio_mes(col_name):
    try:
        # Extraer la parte de la fecha (últimos 6 caracteres)
        fecha = col_name.split()[-1]
        mes = int(fecha[:2])
        anio = int(fecha[2:])
        return (anio, mes)
    except:
        return (0, 0)  # Para manejar posibles errores

# Obtener lista de columnas
cols = df_pivot.columns.tolist()

# Separar en las diferentes categorías
energia_cols = sorted([c for c in cols if c.startswith('Energía KWh')], key=extraer_anio_mes)
ingresos_energia_cols = sorted([c for c in cols if c.startswith('Ingresos Energía USD')], key=extraer_anio_mes)
ingresos_renovables_cols = sorted([c for c in cols if c.startswith('Ingresos Renovables USD')], key=extraer_anio_mes)
ingresos_potencia_cols = sorted([c for c in cols if c.startswith('Ingresos Potencia USD')], key=extraer_anio_mes)

# AQUÍ ESTÁ EL CAMBIO IMPORTANTE: Incluir Energía kWh primero
cols_ordenadas = energia_cols + ingresos_energia_cols + ingresos_renovables_cols + ingresos_potencia_cols

# Reordenar columnas
df_pivot = df_pivot[cols_ordenadas]

# Resetear índice
df_final = df_pivot.reset_index()

# Guardar resultado
df_final.to_excel("./preprocess/serie_ingresos_cronologica.xlsx", index=False)

C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_48132\320038066.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['CENTRAL'] = df['CENTRAL'].replace('nan', pd.NA).fillna(method='ffill')
C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_48132\320038066.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['GENERADOR'] = df['GENERADOR'].replace('nan', pd.NA).fillna(method='ffill')


# 4. Depuracion de serie de tiempo de energia

In [24]:
import pandas as pd
import numpy as np

# Leer el archivo Excel
df = pd.read_excel("./preprocess/serie_ingresos_cronologica.xlsx")

# Lista de agentes a eliminar
agentes_a_eliminar = [
    "TOTAL - CESSA", "TOTAL - CRE R.L.", "TOTAL CRE", "TOTAL - DELAPAZ", 
    "TOTAL - ELFEC", "TOTAL - ENDE", "TOTAL ENDE DELBENI S.A.M.",
    "TOTAL - ENDE DEORURO S.A.", "TOTALES", "Tipo de cambio",
    "TOTAL - SEPSA", "TOTAL - SETAR"
]

# Filtrar y eliminar las filas no deseadas
df = df[~df["CENTRAL"].isin(agentes_a_eliminar)]

# Convertir todas las columnas numéricas (remover comas y convertir a float)
numeric_cols = df.columns.drop(['CENTRAL', 'GENERADOR', 'TECNOLOGIA'])
for col in numeric_cols:
    if df[col].dtype == object:
        df[col] = df[col].str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Identificar periodos únicos (ej: 012023, 022023, etc)
periods = []
for col in df.columns:
    if col.startswith('Energía KWh') or col.startswith('Ingresos Energía USD') or col.startswith('Ingresos Renovables USD') or col.startswith('Ingresos Potencia USD'):
        period = col.split(' ')[-1]
        periods.append(period)

# Calcular precio monómico para cada periodo con el nombre de columna corregido
for period in periods:
    energia_col = f'Energía KWh {period}'
    ing_ener_col = f'Ingresos Energía USD {period}'
    ing_ren_col = f'Ingresos Renovables USD {period}'
    ing_pot_col = f'Ingresos Potencia USD {period}'
    precio_col = f'Precio Monómico USD/MWh {period}'  # Nombre corregido
    
    # Verificar que existen las columnas necesarias
    if all(col in df.columns for col in [energia_col, ing_ener_col, ing_ren_col, ing_pot_col]):
        # Calcular suma de ingresos
        total_ingresos = df[ing_ener_col] + df[ing_ren_col] + df[ing_pot_col]
        
        # Calcular precio monómico evitando división por cero
        df[precio_col] = np.where(
            df[energia_col] > 0,
            (total_ingresos / df[energia_col]) * 1000,  # USD/MWh
            np.nan
        )
    else:
        print(f"Advertencia: Columnas incompletas para el período {period}")

# Guardar el DataFrame con las nuevas columnas
df.to_excel("./data/serie_ingresos.xlsx", index=False)

print("Proceso completado: Se han añadido las columnas de precio monómico con unidades USD/MWh")

Proceso completado: Se han añadido las columnas de precio monómico con unidades USD/MWh


C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_48132\3388279339.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[precio_col] = np.where(
C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_48132\3388279339.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[precio_col] = np.where(
C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_48132\3388279339.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join